In [2]:
import random

class CPABE:
    def __init__(self):
        self.public_params = self.setup()

    def setup(self):
        # 生成公共參數, 在實際系統中會更複雜
        return {'g': random.randint(1, 1000)}

    def key_gen(self, attributes):
        # 為每個屬性生成一個隨機值作為密鑰
        return {attr: random.randint(1, 1000) for attr in attributes}

    def encrypt(self, message, policy):
        # 使用簡單的 XOR 操作模擬加密過程
        ciphertext = message ^ self.public_params['g']
        return {'ciphertext': ciphertext, 'policy': policy}

    def decrypt(self, ciphertext, user_key):
        # 檢查用戶密鑰是否滿足訪問策略
        if self.check_policy(ciphertext['policy'], user_key):
            print("user key: ",user_key)
            # 如果滿足, 使用 XOR 操作解密
            return ciphertext['ciphertext'] ^ self.public_params['g']
        return None

    def check_policy(self, policy, user_key):
        # 遞歸檢查策略是否滿足
        if isinstance(policy, str):
            # 基本情況: 檢查單一屬性
            return policy in user_key
        if policy[0] == 'AND':
            # 所有子策略必須滿足
            return all(self.check_policy(p, user_key) for p in policy[1:])
        if policy[0] == 'OR':
            # 至少一個子策略必須滿足
            print(policy[1:])
            return any(self.check_policy(p, user_key) for p in policy[1:])
        return False

class GradeSystem:
    def __init__(self):
        self.cpabe = CPABE()
        self.grades = {}  # 存儲加密後的成績

    def generate_key(self, attributes):
        # 為用戶生成基於屬性的密鑰
        return self.cpabe.key_gen(attributes)

    def set_grade(self, student, grade, policy):
        # 加密成績並存儲
        encrypted_grade = self.cpabe.encrypt(grade, policy)
        self.grades[student] = encrypted_grade

    def get_grade(self, student, user_key):
        if student not in self.grades:
            return "找不到該學生的成績"
        # 嘗試解密成績
        decrypted_grade = self.cpabe.decrypt(self.grades[student], user_key)
        if decrypted_grade is None:
            return "您沒有權限查看這個成績"
        return decrypted_grade

# 使用示例
grade_system = GradeSystem()

# 生成不同角色的密鑰
teacher_key = grade_system.generate_key(['teacher', 'staff'])
student_a_key = grade_system.generate_key(['student', 'class_A'])
student_b_key = grade_system.generate_key(['student', 'class_B'])

print("密鑰生成:")
print("教師的密鑰:", teacher_key)
print("A班學生的密鑰:", student_a_key)
print("B班學生的密鑰:", student_b_key)

# 設置成績, 允許教師和A班學生查看
policy = ['OR', 'teacher', 'class_A']
grade_system.set_grade("張三", 85, policy)
grade_system.set_grade("李四", 92, policy)

print("\n成績設置完成")

# 不同角色嘗試查看成績
print("\n教師查看成績:")
print("張三的成績:", grade_system.get_grade("張三", teacher_key))
print("李四的成績:", grade_system.get_grade("李四", teacher_key))

print("\nA班學生查看成績:")
print("張三的成績:", grade_system.get_grade("張三", student_a_key))
print("李四的成績:", grade_system.get_grade("李四", student_a_key))

print("\nB班學生嘗試查看成績:")
print("張三的成績:", grade_system.get_grade("張三", student_b_key))
print("李四的成績:", grade_system.get_grade("李四", student_b_key))

密鑰生成:
教師的密鑰: {'teacher': 873, 'staff': 528}
A班學生的密鑰: {'student': 936, 'class_A': 645}
B班學生的密鑰: {'student': 896, 'class_B': 458}

成績設置完成

教師查看成績:
or:  True
user key:  {'teacher': 873, 'staff': 528}
張三的成績: 85
or:  True
user key:  {'teacher': 873, 'staff': 528}
李四的成績: 92

A班學生查看成績:
or:  True
user key:  {'student': 936, 'class_A': 645}
張三的成績: 85
or:  True
user key:  {'student': 936, 'class_A': 645}
李四的成績: 92

B班學生嘗試查看成績:
or:  False
張三的成績: 您沒有權限查看這個成績
or:  False
李四的成績: 您沒有權限查看這個成績
